In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import plotly.express as px
import joblib

In [2]:
data=pd.read_csv("Documents/T1.csv")

In [3]:
data.head()

,Date/Time,LV ActivePower (kW),Wind Speed (m/s),Theoretical_Power_Curve (KWh),Wind Direction (°)
0,01 01 2018 00:00,380.047791,5.311336,416.328908,259.994904
1,01 01 2018 00:10,453.769196,5.672167,519.917511,268.641113
2,01 01 2018 00:20,306.376587,5.216037,390.900016,272.564789
3,01 01 2018 00:30,419.645905,5.659674,516.127569,271.258087
4,01 01 2018 00:40,380.650696,5.577941,491.702972,265.674286


In [4]:
print(data.shape)

(50530, 5)


In [5]:
data['Date/Time'] = pd.to_datetime(data['Date/Time'])

data['YEAR'] = data['Date/Time'].dt.year
data['MONTH'] = data['Date/Time'].dt.month
data['DAY'] = data['Date/Time'].dt.day
data['hour'] = data['Date/Time'].dt.hour

In [6]:
data = data.drop(['Date/Time'], axis=1)

In [7]:
data.head()

,LV ActivePower (kW),Wind Speed (m/s),Theoretical_Power_Curve (KWh),Wind Direction (°),YEAR,MONTH,DAY,hour
0,380.047791,5.311336,416.328908,259.994904,2018,1,1,0
1,453.769196,5.672167,519.917511,268.641113,2018,1,1,0
2,306.376587,5.216037,390.900016,272.564789,2018,1,1,0
3,419.645905,5.659674,516.127569,271.258087,2018,1,1,0
4,380.650696,5.577941,491.702972,265.674286,2018,1,1,0


In [8]:
data.isnull().sum()

LV ActivePower (kW)              0
Wind Speed (m/s)                 0
Theoretical_Power_Curve (KWh)    0
Wind Direction (°)               0
YEAR                             0
MONTH                            0
DAY                              0
hour                             0
dtype: int64

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50530 entries, 0 to 50529
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   LV ActivePower (kW)            50530 non-null  float64
 1   Wind Speed (m/s)               50530 non-null  float64
 2   Theoretical_Power_Curve (KWh)  50530 non-null  float64
 3   Wind Direction (°)             50530 non-null  float64
 4   YEAR                           50530 non-null  int64  
 5   MONTH                          50530 non-null  int64  
 6   DAY                            50530 non-null  int64  
 7   hour                           50530 non-null  int64  
dtypes: float64(4), int64(4)
memory usage: 3.1 MB


In [10]:
out=[]
def iqr_outliers(df):
    q1 = df.quantile(0.25)
    q3 = df.quantile(0.75)
    iqr = q3-q1
    Lower_tail = q1 - 1.5 * iqr
    Upper_tail = q3 + 1.5 * iqr
    for i in df:
        if i > Upper_tail or i < Lower_tail:
            out.append(i)
    print(len(out))

iqr_outliers(data['LV ActivePower (kW)'])
iqr_outliers(data['Theoretical_Power_Curve (KWh)'])
iqr_outliers(data['Wind Direction (°)'])

0
0
0


In [11]:
from sklearn.model_selection import train_test_split

In [12]:
X= data.drop(['Wind Speed (m/s)'] , axis = 1).values
y= data['Wind Speed (m/s)' ].values

In [13]:
x_train , x_test , y_train , y_test = train_test_split(X,y , test_size= 0.2 , random_state=42)

In [15]:
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeRegressor
Regressor = DecisionTreeRegressor()
Regressor.fit(x_train, y_train)
yhat = Regressor.predict(x_test)
mae = mean_absolute_error(y_test, yhat)
print('MAE: %.3f' % mae)

MAE: 0.165


In [16]:
from sklearn.metrics import r2_score
R2_Decision_Tree = r2_score(y_test,yhat)
print('R2: %.3f' %R2_Decision_Tree)

R2: 0.986


In [17]:
print(Regressor.score(x_train, y_train))

0.9999996539579773


In [18]:
ACC_Decision=Regressor.score(x_test, y_test)
print(Regressor.score(x_test, y_test))

0.9861239296060794


In [27]:
joblib.dump(Regressor, "power_prediction.sav")

['power_prediction.sav']